In [ ]:
from keras import Model
from keras.layers import Input, Conv2D, MaxPool2D, Dropout
from keras.layers import concatenate, UpSampling2D

class U_Net(Model) : 
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    
    
    pass